# STAT 857 - W23 Project 2
## Evan Callaghan | April 17, 2023

### 1. Configuring setup
Enter brief description of this section here...

In [ ]:
pip install lightgbm xgboost reverse_geocoder haversine optuna

In [114]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import optuna
from haversine import haversine
import reverse_geocoder as revgc
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.feature_selection import RFECV, RFE
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier

pd.set_option('display.max_columns', None, 'display.max_rows', None)

### 2. Data Exploration Section

enter brief description of this section here...

In [115]:
## Reading the data
train = pd.read_csv('Data/W23P2_train.csv')
test = pd.read_csv('Data/W23P2_test.csv')
sub = pd.read_csv('Data/Sample_Submission.csv')

In [116]:
train.head()

,interest_level,bathrooms,bedrooms,latitude,longitude,price,street_address,Dining.Room,Pre.War,Laundry.in.Building,Dishwasher,Hardwood.Floors,Dogs.Allowed,Cats.Allowed,Doorman,Elevator,No.Fee,Laundry.in.Unit,Swimming.Pool,Roof.Deck,Fitness.Center,High.Speed.Internet,Wheelchair.Access,Reduced.Fee,Common.Outdoor.Space,Parking.Space,Laundry.In.Building,Concierge,PublicOutdoor,Garage,Laundry.In.Unit,New.Construction,Exclusive,Private.Outdoor.Space,Garden.Patio,Outdoor.Space,Common.roof.deck,Fireplace,Storage,dishwasher,Newly.renovated,No.pets,prewar,LOWRISE,LAUNDRY,SIMPLEX,HARDWOOD,Balcony,Furnished,Terrace,High.ceilings,Loft,High.Ceilings,Hardwood.floors,Stainless.Steel.Appliances,Central.A.C,Multi.Level,Garden,View,Concierge.Service,Prewar,Patio,Roof.deck,Laundry.Room,elevator,pool,Full.time.doorman,Gym.Fitness,Pool,LIVE.IN.SUPER,HIGH.CEILINGS,Washer.in.Unit,Dryer.in.Unit,Green.Building,Short.Term.Allowed,Subway,Laundry,Hardwood,Renovated,Marble.Bath,Granite.Kitchen,Light,Pet.Friendly,Guarantors.Accepted,storage,EXPOSED.BRICK,Air.conditioning,Live.in.superintendent,On.site.laundry,High.Ceiling,WiFi,Lounge,Actual.Apt..Photos,Skylight,Parking,EAT.IN.KITCHEN,Sauna,Live.Work,Duplex,Pets.on.approval,NO.FEE,Eat.In.Kitchen,ACTUAL.APT..PHOTOS,X24.7.Concierge,Virtual.Doorman,On.site.Garage,Valet,renovated,New.construction,Bike.room,HIGHRISE,ROOFDECK,private.outdoor.space,balcony,Walk.in.Closet.s.,On.site.Laundry,assigned.parking.space,Private.parking,Washer.Dryer,Luxury.building,Post.War,Indoor.Pool,Residents.Lounge,WiFi.Access,Valet.Parking,Common.parking.Garage,Courtyard,Lounge.room,Childrens.Playroom,Residents.Garden,Marble.Bathroom,FT.Doorman,Hi.Rise,Basement.Storage,Full.Service.Garage,Health.Club,Washer.Dryer.in.building,Children.s.Playroom,Outdoor.Entertainment.Space,In.Unit.Washer.Dryer,Bike.Room,Live.In.Superintendent,Live.in.Super,doorman,On.site.Parking.Lot,Cable.Satellite.TV,Microwave,Outdoor.Areas,Decorative.Fireplace,terrace,Gym,Sublet,Live.In.Super,Wheelchair.Ramp,private.balcony,Private.balcony,Common.backyard,Washer...Dryer,Multi.level,patio,Exposed.Brick,building.common.outdoor.space,garden,Washer.Dryer.in.Unit,Private.Deck,Shares.OK,gym,Private.outdoor.space,Gym.In.Building,Common.garden
0,medium,1.0,4,40.7831,-73.9449,3675,1962 First Avenue,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,medium,1.0,3,40.7542,-73.9724,3600,155 East 47th Street,1,0,1,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,medium,1.0,2,40.7285,-73.9790,2900,206 Avenue B,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,medium,2.0,4,40.7306,-73.9837,7200,352 E 13 St,1,1,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,high,1.0,1,40.7645,-73.9840,2500,260 W 54 St.,0,0,1,1,1,1,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [117]:
print(train.shape)
print(test.shape)

(9846, 170)
(6564, 170)


In [118]:
train['interest_level'].value_counts()

low       3884
medium    3716
high      2246
Name: interest_level, dtype: int64

In [119]:
train.describe()

,bathrooms,bedrooms,latitude,longitude,price,Dining.Room,Pre.War,Laundry.in.Building,Dishwasher,Hardwood.Floors,Dogs.Allowed,Cats.Allowed,Doorman,Elevator,No.Fee,Laundry.in.Unit,Swimming.Pool,Roof.Deck,Fitness.Center,High.Speed.Internet,Wheelchair.Access,Reduced.Fee,Common.Outdoor.Space,Parking.Space,Laundry.In.Building,Concierge,PublicOutdoor,Garage,Laundry.In.Unit,New.Construction,Exclusive,Private.Outdoor.Space,Garden.Patio,Outdoor.Space,Common.roof.deck,Fireplace,Storage,dishwasher,Newly.renovated,No.pets,prewar,LOWRISE,LAUNDRY,SIMPLEX,HARDWOOD,Balcony,Furnished,Terrace,High.ceilings,Loft,High.Ceilings,Hardwood.floors,Stainless.Steel.Appliances,Central.A.C,Multi.Level,Garden,View,Concierge.Service,Prewar,Patio,Roof.deck,Laundry.Room,elevator,pool,Full.time.doorman,Gym.Fitness,Pool,LIVE.IN.SUPER,HIGH.CEILINGS,Washer.in.Unit,Dryer.in.Unit,Green.Building,Short.Term.Allowed,Subway,Laundry,Hardwood,Renovated,Marble.Bath,Granite.Kitchen,Light,Pet.Friendly,Guarantors.Accepted,storage,EXPOSED.BRICK,Air.conditioning,Live.in.superintendent,On.site.laundry,High.Ceiling,WiFi,Lounge,Actual.Apt..Photos,Skylight,Parking,EAT.IN.KITCHEN,Sauna,Live.Work,Duplex,Pets.on.approval,NO.FEE,Eat.In.Kitchen,ACTUAL.APT..PHOTOS,X24.7.Concierge,Virtual.Doorman,On.site.Garage,Valet,renovated,New.construction,Bike.room,HIGHRISE,ROOFDECK,private.outdoor.space,balcony,Walk.in.Closet.s.,On.site.Laundry,assigned.parking.space,Private.parking,Washer.Dryer,Luxury.building,Post.War,Indoor.Pool,Residents.Lounge,WiFi.Access,Valet.Parking,Common.parking.Garage,Courtyard,Lounge.room,Childrens.Playroom,Residents.Garden,Marble.Bathroom,FT.Doorman,Hi.Rise,Basement.Storage,Full.Service.Garage,Health.Club,Washer.Dryer.in.building,Children.s.Playroom,Outdoor.Entertainment.Space,In.Unit.Washer.Dryer,Bike.Room,Live.In.Superintendent,Live.in.Super,doorman,On.site.Parking.Lot,Cable.Satellite.TV,Microwave,Outdoor.Areas,Decorative.Fireplace,terrace,Gym,Sublet,Live.In.Super,Wheelchair.Ramp,private.balcony,Private.balcony,Common.backyard,Washer...Dryer,Multi.level,patio,Exposed.Brick,building.common.outdoor.space,garden,Washer.Dryer.in.Unit,Private.Deck,Shares.OK,gym,Private.outdoor.space,Gym.In.Building,Common.garden
count,9846.000000,9846.000000,9846.000000,9846.000000,9.846000e+03,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.0,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.000000,9846.

#### Data Cleaning

In [120]:
## Removing training observations with prices higher than $40,000
train = train[train['price'] < 40000].reset_index(drop = True)

## Creating columns for outlier locations
train['location_outlier'] = np.where(train['latitude'] < 35, 1, 0)
test['location_outlier'] = np.where(test['latitude'] < 35, 1, 0)

## Log transformation on the price variable
train['price'] = np.log(train['price'])
test['price'] = np.log(test['price'])

In [59]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA().fit(X)

print(pca.explained_variance_ratio_)

In [ ]:
X_pca = pca.components_

X_pca.head()

In [121]:
## Condensing redundant variables

laundry_vars = ['Laundry.in.Building', 'Laundry.in.Unit', 'Laundry.In.Building', 'Laundry.In.Unit', 'LAUNDRY', 'Washer.in.Unit', 
                'Dryer.in.Unit', 'Laundry.Room', 'Laundry', 'On.site.laundry', 'On.site.Laundry', 'Washer.Dryer', 'Washer.Dryer.in.building', 
                'In.Unit.Washer.Dryer', 'Washer...Dryer', 'Washer.Dryer.in.Unit']

parking_vars = ['Parking.Space', 'Garage', 'Parking', 'On.site.Garage', 'assigned.parking.space', 'Common.parking.Garage', 'Full.Service.Garage', 
               'On.site.Parking.Lot', 'Private.parking', 'Valet.Parking', 'Valet']

deck_vars = ['Roof.Deck', 'Balcony', 'Terrace', 'Patio', 'Roof.deck', 'balcony', 'terrace', 'patio', 'private.balcony', 'Private.balcony', 
             'Private.Deck', 'Common.roof.deck', 'ROOFDECK']

outdoor_vars = ['Courtyard', 'Outdoor.Entertainment.Space', 'Common.garden', 'Private.Outdoor.Space', 'private.outdoor.space', 'Private.outdoor.space', 
               'garden', 'Common.Outdoor.Space', 'PublicOutdoor', 'Garden.Patio', 'Outdoor.Space', 'Garden', 'Outdoor.Areas', 'Common.backyard', 
                'building.common.outdoor.space', 'Residents.Garden']

dishwasher_vars = ['Dishwasher', 'dishwasher']
gym_vars = ['Fitness.Center', 'Gym.Fitness', 'Health.Club', 'Gym', 'gym', 'Gym.In.Building']
pool_vars = ['Swimming.Pool', 'Pool', 'pool', 'Indoor.Pool']
elevator_vars = ['Elevator', 'elevator']
storage_vars = ['Storage', 'storage', 'Basement.Storage']
internet_vars = ['High.Speed.Internet', 'WiFi', 'WiFi.Access']
bike_vars = ['Bike.room', 'Bike.Room']
pet_friendly_vars = ['Dogs.Allowed', 'Cats.Allowed', 'Pet.Friendly', 'Pets.on.approval']
service_vars = ['Doorman', 'Concierge', 'Concierge.Service', 'Full.time.doorman', 'X24.7.Concierge', 'Virtual.Doorman', 'FT.Doorman', 'doorman']
super_vars = ['LIVE.IN.SUPER', 'Live.in.superintendent', 'Live.In.Superintendent','Live.in.Super', 'Live.In.Super']
hardwood_vars = ['Hardwood.Floors', 'HARDWOOD', 'Hardwood.floors', 'Hardwood']
ceiling_vars = ['High.ceilings', 'High.Ceilings', 'HIGH.CEILINGS', 'High.Ceiling']
brick_vars = ['EXPOSED.BRICK', 'Exposed.Brick']
construction_vars = ['New.Construction', 'Newly.renovated','Renovated', 'renovated', 'New.construction']
photo_vars = ['Actual.Apt..Photos', 'ACTUAL.APT..PHOTOS']
lounge_vars = ['Residents.Lounge', 'Lounge.room', 'Lounge']
playroom_vars = ['Childrens.Playroom', 'Children.s.Playroom']
ac_vars = ['Central.A.C', 'Air.conditioning']
kitchen_vars = ['EAT.IN.KITCHEN','Eat.In.Kitchen', 'Granite.Kitchen']
pricing_vars = ['No.Fee', 'NO.FEE']
accessibity_vars = ['Wheelchair.Ramp', 'Wheelchair.Access']
multi_level_vars = ['Multi.Level', 'Multi.level']
fireplace_vars = ['Fireplace', 'Decorative.Fireplace']
highrise_vars = ['Hi.Rise', 'HIGHRISE']
marble_bath_vars = ['Marble.Bath', 'Marble.Bathroom']
prewar_vars = ['Pre.War', 'prewar', 'Prewar']

## Condensing training data
train['Has_Laundry'] = np.where(np.sum(train[laundry_vars], axis = 1) > 0, 1, 0)
train['Has_Dishwasher'] = np.where(np.sum(train[dishwasher_vars], axis = 1) > 0, 1, 0)
train['Has_Gym'] = np.where(np.sum(train[gym_vars], axis = 1) > 0, 1, 0)
train['Has_pool'] = np.where(np.sum(train[pool_vars], axis = 1) > 0, 1, 0)
train['Has_Elevator'] = np.where(np.sum(train[elevator_vars], axis = 1) > 0, 1, 0)
train['Has_storage'] = np.where(np.sum(train[storage_vars], axis = 1) > 0, 1, 0)
train['Has_Wifi'] = np.where(np.sum(train[internet_vars], axis = 1) > 0, 1, 0)
train['Has_Bike'] = np.where(np.sum(train[bike_vars], axis = 1) > 0, 1, 0)
train['Has_Parking'] = np.where(np.sum(train[parking_vars], axis = 1) > 0, 1, 0)
train['Has_Deck'] = np.where(np.sum(train[deck_vars], axis = 1) > 0, 1, 0)
train['Has_Lounge'] = np.where(np.sum(train[lounge_vars], axis = 1) > 0, 1, 0)
train['Has_Playroom'] = np.where(np.sum(train[playroom_vars], axis = 1) > 0, 1, 0)
train['Has_AC'] = np.where(np.sum(train[ac_vars], axis = 1) > 0, 1, 0)
train['Has_Kitchen'] = np.where(np.sum(train[kitchen_vars], axis = 1) > 0, 1, 0)
train['No_Fee'] = np.where(np.sum(train[pricing_vars], axis = 1) > 0, 1, 0)
train['Outdoor_Area'] = np.where(np.sum(train[outdoor_vars], axis = 1) > 0, 1, 0)
train['Pet_Friendly'] = np.where(np.sum(train[pet_friendly_vars], axis = 1) > 0, 1, 0)
train['Has_Services'] = np.where(np.sum(train[service_vars], axis = 1) > 0, 1, 0)
train['Super.'] = np.where(np.sum(train[super_vars], axis = 1) > 0, 1, 0)
train['Hardwood_Floor'] = np.where(np.sum(train[hardwood_vars], axis = 1) > 0, 1, 0)
train['High_Ceilings'] = np.where(np.sum(train[ceiling_vars], axis = 1) > 0, 1, 0)
train['Brick'] = np.where(np.sum(train[brick_vars], axis = 1) > 0, 1, 0)
train['Newly_Renovated'] = np.where(np.sum(train[construction_vars], axis = 1) > 0, 1, 0)
train['Has_Photos'] = np.where(np.sum(train[photo_vars], axis = 1) > 0, 1, 0)
train['Accessible'] = np.where(np.sum(train[accessibity_vars], axis = 1) > 0, 1, 0)
train['Multi_Level'] = np.where(np.sum(train[multi_level_vars], axis = 1) > 0, 1, 0)
train['Fire'] = np.where(np.sum(train[fireplace_vars], axis = 1) > 0, 1, 0)
train['Highrise'] = np.where(np.sum(train[highrise_vars], axis = 1) > 0, 1, 0)
train['Marble_Bath'] = np.where(np.sum(train[marble_bath_vars], axis = 1) > 0, 1, 0)
train['Pre_War'] = np.where(np.sum(train[prewar_vars], axis = 1) > 0, 1, 0)

## Condensing testing data
test['Has_Laundry'] = np.where(np.sum(test[laundry_vars], axis = 1) > 0, 1, 0)
test['Has_Dishwasher'] = np.where(np.sum(test[dishwasher_vars], axis = 1) > 0, 1, 0)
test['Has_Gym'] = np.where(np.sum(test[gym_vars], axis = 1) > 0, 1, 0)
test['Has_pool'] = np.where(np.sum(test[pool_vars], axis = 1) > 0, 1, 0)
test['Has_Elevator'] = np.where(np.sum(test[elevator_vars], axis = 1) > 0, 1, 0)
test['Has_storage'] = np.where(np.sum(test[storage_vars], axis = 1) > 0, 1, 0)
test['Has_Wifi'] = np.where(np.sum(test[internet_vars], axis = 1) > 0, 1, 0)
test['Has_Bike'] = np.where(np.sum(test[bike_vars], axis = 1) > 0, 1, 0)
test['Has_Parking'] = np.where(np.sum(test[parking_vars], axis = 1) > 0, 1, 0)
test['Has_Deck'] = np.where(np.sum(test[deck_vars], axis = 1) > 0, 1, 0)
test['Has_Lounge'] = np.where(np.sum(test[lounge_vars], axis = 1) > 0, 1, 0)
test['Has_Playroom'] = np.where(np.sum(test[playroom_vars], axis = 1) > 0, 1, 0)
test['Has_AC'] = np.where(np.sum(test[ac_vars], axis = 1) > 0, 1, 0)
test['Has_Kitchen'] = np.where(np.sum(test[kitchen_vars], axis = 1) > 0, 1, 0)
test['No_Fee'] = np.where(np.sum(test[pricing_vars], axis = 1) > 0, 1, 0)
test['Outdoor_Area'] = np.where(np.sum(test[outdoor_vars], axis = 1) > 0, 1, 0)
test['Pet_Friendly'] = np.where(np.sum(test[pet_friendly_vars], axis = 1) > 0, 1, 0)
test['Has_Services'] = np.where(np.sum(test[service_vars], axis = 1) > 0, 1, 0)
test['Super.'] = np.where(np.sum(test[super_vars], axis = 1) > 0, 1, 0)
test['Hardwood_Floor'] = np.where(np.sum(test[hardwood_vars], axis = 1) > 0, 1, 0)
test['High_Ceilings'] = np.where(np.sum(test[ceiling_vars], axis = 1) > 0, 1, 0)
test['Brick'] = np.where(np.sum(test[brick_vars], axis = 1) > 0, 1, 0)
test['Newly_Renovated'] = np.where(np.sum(test[construction_vars], axis = 1) > 0, 1, 0)
test['Has_Photos'] = np.where(np.sum(test[photo_vars], axis = 1) > 0, 1, 0)
test['Accessible'] = np.where(np.sum(test[accessibity_vars], axis = 1) > 0, 1, 0)
test['Multi_Level'] = np.where(np.sum(test[multi_level_vars], axis = 1) > 0, 1, 0)
test['Fire'] = np.where(np.sum(test[fireplace_vars], axis = 1) > 0, 1, 0)
test['Highrise'] = np.where(np.sum(test[highrise_vars], axis = 1) > 0, 1, 0)
test['Marble_Bath'] = np.where(np.sum(test[marble_bath], axis = 1) > 0, 1, 0)
test['Pre_War'] = np.where(np.sum(test[prewar_vars], axis = 1) > 0, 1, 0)

to_drop = [laundry_vars, dishwasher_vars, gym_vars, pool_vars, elevator_vars, storage_vars, internet_vars, bike_vars, 
           parking_vars, deck_vars, lounge_vars, playroom_vars, ac_vars, kitchen_vars, pricing_vars, outdoor_vars, 
           pet_friendly_vars, service_vars, super_vars, hardwood_vars, ceiling_vars, brick_vars, construction_vars, photo_vars, 
           accessibity_vars, multi_level_vars, fireplace_vars, highrise_vars, marble_bath, prewar_vars] 

for cols in to_drop:
    train = train.drop(columns = cols)
    test = test.drop(columns = cols)

#### Variable Engineering

In [122]:
amenities = ['Has_Laundry', 'Has_Dishwasher', 'Has_Gym', 'Has_pool', 'Has_Elevator', 'Has_storage', 'Has_Wifi', 'Has_Bike', 
             'Has_Parking', 'Has_Deck', 'Has_Lounge', 'Has_Playroom', 'Has_AC', 'Has_Kitchen', 'Outdoor_Area', 'Furnished', 
             'Stainless.Steel.Appliances', 'Cable.Satellite.TV', 'Microwave', 'Sauna']

X_train_amenities = train[amenities]
Y_train_amenities = train['interest_level']

X_test_amenities = test[amenities]

## Building the K-NN amentity model
knn_amentity_md = KNeighborsClassifier(n_neighbors = 15).fit(X_train_amenities, Y_train_amenities)

## Predicting on the training and testing sets
train_preds = knn_amentity_md.predict_proba(X_train_amenities)
test_preds = knn_amentity_md.predict_proba(X_test_amenities)

## Extracting the likelihood of high interest
train['Amenity_Score'] = pd.DataFrame(train_preds)[0]
test['Amenity_Score'] = pd.DataFrame(test_preds)[0]

In [123]:
qualities = ['Exclusive', 'View', 'Green.Building', 'Light', 'Skylight', 'Walk.in.Closet.s.', 'Luxury.building', 
             'Hardwood_Floor', 'High_Ceilings', 'Brick', 'Newly_Renovated', 'Multi_Level', 'Highrise', 'Marble_Bath', 'Fire']

X_train_qualities = train[qualities]
Y_train_qualities = train['interest_level']

X_test_qualities = test[qualities]

## Building the K-NN amentity model
knn_quality_md = KNeighborsClassifier(n_neighbors = 15).fit(X_train_qualities, Y_train_qualities)

## Predicting on the training and testing sets
train_preds = knn_quality_md.predict_proba(X_train_qualities)
test_preds = knn_quality_md.predict_proba(X_test_qualities)

## Extracting the likelihood of high interest
train['Quality_Score'] = pd.DataFrame(train_preds)[0]
test['Quality_Score'] = pd.DataFrame(test_preds)[0]

In [124]:
## Computing distance from unit to Central Park
def central_park(data):
    data['distance'] = np.nan
    for i in range(data.shape[0]):
        lat = data.at[i, 'latitude']; long = data.at[i, 'longitude']
        data.at[i, 'distance'] = haversine((lat, long), (40.781179, -73.966098), unit = 'mi')
    return data

## Applying function
train = central_park(train)
test = central_park(test)

In [125]:
## Price per bedroom
rooms_train = train['bedrooms'].apply(lambda x: max(x, 0.5))
train['price_per_bedroom'] = train['price'] / rooms_train

rooms_test = test['bedrooms'].apply(lambda x: max(x, 0.5))
test['price_per_bedroom'] = test['price'] / rooms_test

In [126]:
## Standardizing numeric variables
def min_max_scaler(data, minimun, maximum):
    data_scaled = (data - minimun) / (maximum - minimun)
    return data_scaled

train['price'] = min_max_scaler(train['price'], np.min(train['price']), np.max(train['price']))
train['bathrooms'] = min_max_scaler(train['bathrooms'], np.min(train['bathrooms']), np.max(train['bathrooms']))
train['bedrooms'] = min_max_scaler(train['bedrooms'], np.min(train['bedrooms']), np.max(train['bedrooms']))
train['distance'] = min_max_scaler(train['distance'], np.min(train['distance']), np.max(train['distance']))
train['price_per_bedroom'] = min_max_scaler(train['price_per_bedroom'], np.min(train['price_per_bedroom']), np.max(train['price_per_bedroom']))
train['latitude'] = min_max_scaler(train['latitude'], np.min(train['latitude']), np.max(train['latitude']))
train['longitude'] = min_max_scaler(train['longitude'], np.min(train['longitude']), np.max(train['longitude']))


test['price'] = min_max_scaler(test['price'], np.min(train['price']), np.max(train['price']))
test['bathrooms'] = min_max_scaler(test['bathrooms'], np.min(train['bathrooms']), np.max(train['bathrooms']))
test['bedrooms'] = min_max_scaler(test['bedrooms'], np.min(train['bedrooms']), np.max(train['bedrooms']))
test['distance'] = min_max_scaler(test['distance'], np.min(train['distance']), np.max(train['distance']))
test['price_per_bedroom'] = min_max_scaler(test['price_per_bedroom'], np.min(train['price_per_bedroom']), np.max(train['price_per_bedroom']))
test['latitude'] = min_max_scaler(test['latitude'], np.min(train['latitude']), np.max(train['latitude']))
test['longitude'] = min_max_scaler(test['longitude'], np.min(train['longitude']), np.max(train['longitude']))

In [127]:
train = train[['interest_level', 'bathrooms', 'bedrooms', 'latitude', 'longitude', 'price', 'Dining.Room', 'Reduced.Fee', 'Exclusive', 
               'LOWRISE', 'SIMPLEX', 'Furnished', 'Loft', 'Stainless.Steel.Appliances', 'View', 'Green.Building', 'Short.Term.Allowed', 
               'Subway', 'Light', 'Guarantors.Accepted', 'Skylight', 'Sauna', 'Live.Work', 'Duplex', 'Walk.in.Closet.s.', 'Luxury.building', 
               'Post.War', 'Cable.Satellite.TV', 'Microwave', 'Sublet', 'Shares.OK', 'Has_Laundry', 'Has_Dishwasher', 'Has_Gym', 'Has_pool', 
               'Has_Elevator', 'Has_storage', 'Has_Wifi', 'Has_Bike', 'Has_Parking', 'Has_Deck', 'Has_Lounge', 'Has_Playroom', 'Has_AC', 
               'Has_Kitchen', 'No_Fee', 'Outdoor_Area', 'Pet_Friendly', 'Has_Services', 'Super.', 'Hardwood_Floor', 'High_Ceilings', 'Brick', 
               'Newly_Renovated', 'Has_Photos', 'Accessible', 'Multi_Level', 'Fire', 'Highrise', 'Marble_Bath', 'Pre_War', 'Amenity_Score', 
               'Quality_Score', 'distance', 'price_per_bedroom', 'location_outlier']]

test = test[['ID', 'bathrooms', 'bedrooms', 'latitude', 'longitude', 'price', 'Dining.Room', 'Reduced.Fee', 'Exclusive', 
             'LOWRISE', 'SIMPLEX', 'Furnished', 'Loft', 'Stainless.Steel.Appliances', 'View', 'Green.Building', 'Short.Term.Allowed', 
             'Subway', 'Light', 'Guarantors.Accepted', 'Skylight', 'Sauna', 'Live.Work', 'Duplex', 'Walk.in.Closet.s.', 'Luxury.building', 
             'Post.War', 'Cable.Satellite.TV', 'Microwave', 'Sublet', 'Shares.OK', 'Has_Laundry', 'Has_Dishwasher', 'Has_Gym', 'Has_pool', 
             'Has_Elevator', 'Has_storage', 'Has_Wifi', 'Has_Bike', 'Has_Parking', 'Has_Deck', 'Has_Lounge', 'Has_Playroom', 'Has_AC', 
             'Has_Kitchen', 'No_Fee', 'Outdoor_Area', 'Pet_Friendly', 'Has_Services', 'Super.', 'Hardwood_Floor', 'High_Ceilings', 'Brick', 
             'Newly_Renovated', 'Has_Photos', 'Accessible', 'Multi_Level', 'Fire', 'Highrise', 'Marble_Bath', 'Pre_War', 'Amenity_Score', 
             'Quality_Score', 'distance', 'price_per_bedroom', 'location_outlier']]

In [128]:
train.head()

,interest_level,bathrooms,bedrooms,latitude,longitude,price,Dining.Room,Reduced.Fee,Exclusive,LOWRISE,SIMPLEX,Furnished,Loft,Stainless.Steel.Appliances,View,Green.Building,Short.Term.Allowed,Subway,Light,Guarantors.Accepted,Skylight,Sauna,Live.Work,Duplex,Walk.in.Closet.s.,Luxury.building,Post.War,Cable.Satellite.TV,Microwave,Sublet,Shares.OK,Has_Laundry,Has_Dishwasher,Has_Gym,Has_pool,Has_Elevator,Has_storage,Has_Wifi,Has_Bike,Has_Parking,Has_Deck,Has_Lounge,Has_Playroom,Has_AC,Has_Kitchen,No_Fee,Outdoor_Area,Pet_Friendly,Has_Services,Super.,Hardwood_Floor,High_Ceilings,Brick,Newly_Renovated,Has_Photos,Accessible,Multi_Level,Fire,Highrise,Marble_Bath,Pre_War,Amenity_Score,Quality_Score,distance,price_per_bedroom,location_outlier
0,medium,0.181818,0.666667,0.914341,0.016393,0.489611,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.200000,0.266667,0.000164,0.033537,0
1,medium,0.181818,0.500000,0.913693,0.016027,0.485054,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0.133333,0.333333,0.000309,0.072919,0
2,medium,0.181818,0.333333,0.913117,0.015940,0.437266,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.200000,0.333333,0.000644,0.145995,0
3,medium,0.363636,0.666667,0.913164,0.015877,0.638245,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0.200000,0.333333,0.000628,0.043314,0
4,high,0.181818,0.166667,0.913924,0.015873,0.404464,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0.266667,0.333333,0.000233,0.369169,0


#### Splitting the training data into train and validation sets

In [129]:
## Defining input and target variables
X = train.drop(columns = ['interest_level'])
Y = train['interest_level']

## Splitting the data
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state = 42)

## Saving new csv files
pd.concat([X_train, Y_train], axis = 1).to_csv('Data/Training.csv', index = False)
pd.concat([X_validation, Y_validation], axis = 1).to_csv('Data/Validation.csv', index = False)
test.to_csv('Data/Testing.csv', index = False)

#### Feature Selection

In [ ]:
## Using RFE with RandomForestClassifer to identify most important features
def flat_list(my_list):
    
    ## Defining list to store results
    out_list = list()
    for i in my_list:
        out_list += i
    return out_list

def RF_RFE_rep_cross_val(X, Y, numb_folds, max_features, numb_reps):
    
    ## Defining list to store results
    RFE_rep_results = list()
    for i in range(2, max_features):
        RFE_rep_results.append(RF_rep_cross_val(X, Y, numb_folds, i, numb_reps))
        print('Features -->', i) ## Sanity check
    return RFE_rep_results

def RF_rep_cross_val(X, Y, numb_folds, numb_features, numb_reps):
    
    ## Defining the list to store results
    rep_results = list()
    for i in range(0, numb_reps):
        rep_results.append(RF_cross_val(X, Y, numb_folds, numb_features))
    return flat_list(rep_results)

def RF_cross_val(X, Y, numb_folds, numb_features):
    
    ## Defining list to store results
    results = list()
    
    ## Defining the number of folds
    kf = KFold(n_splits = numb_folds, shuffle = True)
    
    for train_index, test_index in kf.split(X):
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
        
        ## Running RFE with i features
        RF_rfe = RFE(estimator = RandomForestClassifier(n_estimators = 100, max_depth = 5), 
                     n_features_to_select = numb_features).fit(X_train, Y_train)
        
        ## Variables to be considered
        to_select = X_train.columns[RF_rfe.support_]
        to_select_list.append(RF_rfe.support_)
        
        ## Building the Random Forest model
        X_train_md = X_train[to_select]
        X_test_md = X_test[to_select]
        
        RF_md = RandomForestClassifier(n_estimators = 100, max_depth = 5).fit(X_train_md, Y_train)
        
        ## Predicting on the test data-frame and storing RMSE
        results.append(log_loss(Y_test, RF_md.predict_proba(X_test_md)))

    return results

## Defining list to store results
to_select_list = list()

## Defining input and target variables
data = pd.read_csv('Data/Training.csv')
X = data.drop(columns = ['interest_level']); Y = data['interest_level']

## Running RFE to estimate number of features to be selected
RFE_numb_features = RF_RFE_rep_cross_val(X, Y, numb_folds = 3, max_features = 50, numb_reps = 1)

In [131]:
## Identifying features
features = pd.DataFrame(to_select_list)
features.columns = X.columns
feature_selections = 100 * features.apply(np.sum, axis = 0) / features.shape[0]
feature_selections = pd.DataFrame(feature_selections).reset_index(drop = False)

## Model performance given the number of variables
feature_performance = pd.DataFrame(RFE_numb_features)
feature_performance.columns = [['Split_1', 'Split_2', 'Split_3']]
feature_performance['Mean'] = feature_performance.apply(np.mean, axis = 1)
feature_performance['Num_features'] = feature_performance.index + 2

In [151]:
feature_performance.head(10)

,Split_1,Split_2,Split_3,Mean,Num_features
0,0.942419,0.943362,0.954969,0.946916,2
1,0.926649,0.930962,0.944421,0.934011,3
2,0.931439,0.917823,0.922789,0.924017,4
3,0.920113,0.936625,0.934232,0.930324,5
4,0.924029,0.929670,0.928566,0.927421,6
5,0.952240,0.923908,0.933497,0.936548,7
6,0.937834,0.947906,0.934465,0.940068,8
7,0.932118,0.933907,0.930176,0.932067,9
8,0.930252,0.934796,0.939120,0.934723,10
9,0.931496,0.927754,0.941887,0.933712,11


In [149]:
feature_performance

,Split_1,Split_2,Split_3,Mean,Num_features
0,0.942419,0.943362,0.954969,0.946916,2
1,0.926649,0.930962,0.944421,0.934011,3
2,0.931439,0.917823,0.922789,0.924017,4
3,0.920113,0.936625,0.934232,0.930324,5
4,0.924029,0.929670,0.928566,0.927421,6
5,0.952240,0.923908,0.933497,0.936548,7
6,0.937834,0.947906,0.934465,0.940068,8
7,0.932118,0.933907,0.930176,0.932067,9
8,0.930252,0.934796,0.939120,0.934723,10
9,0.931496,0.927754,0.941887,0.933712,11


In [150]:
feature_selections.sort_values(0, ascending = False).head(10)

,index,0
4,price,100.000000
63,price_per_bedroom,100.000000
44,No_Fee,96.527778
62,distance,95.833333
3,longitude,93.055556
2,latitude,89.583333
61,Quality_Score,89.583333
49,Hardwood_Floor,89.583333
60,Amenity_Score,86.805556
1,bedrooms,83.333333


The model which used to top six features had the lowest cross-validation score, so we will consider the top six features for the final modelling processes.

#### Hyper-Parameter Tuning

In [153]:
## Reading the data
train = pd.read_csv('Data/Training.csv')
val = pd.read_csv('Data/Validation.csv')

## Defining the input and target variables
X_train = train.drop(columns = ['interest_level']); Y_train = train['interest_level']
X_validation = val.drop(columns = ['interest_level']); Y_validation = val['interest_level']

In [154]:
def rf_objective(trial):

    ## Defining the XGBoost hyper-parameter grid
    rf_param_grid = {'n_estimators': trial.suggest_int('n_estimators', 100, 1000, 50),
                     'max_depth': trial.suggest_int('max_depth', 3, 12), 
                     'min_samples_split': trial.suggest_int('min_samples_split', 2, 20), 
                     'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20), 
                     'random_state': trial.suggest_int('random_state', 1, 500),
                     'max_features': trial.suggest_categorical('max_features', ['sqrt', None])
                    }
    
    ## Building the model
    rf_md = RandomForestClassifier(**rf_param_grid, criterion = 'log_loss', n_jobs = -1).fit(X_train, Y_train)
    
    ## Predicting on the test data-frame
    rf_md_preds = rf_md.predict_proba(X_validation)
    
    ## Evaluating model performance on the test set
    rf_md_error = log_loss(Y_validation, rf_md_preds)
    
    return rf_md_error

def xgb_objective(trial):

    ## Defining the XGBoost hyper-parameter grid
    xgboost_param_grid = {'n_estimators': trial.suggest_int('n_estimators', 100, 1000, 50), 
                          'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, step = 0.01), 
                          'max_depth': trial.suggest_int('max_depth', 3, 15), 
                          'gamma': trial.suggest_float('gamma', 0, 0.3, step = 0.05), 
                          'min_child_weight': trial.suggest_int('min_child_weight', 1, 20), 
                          'subsample': trial.suggest_float('subsample', 0.6, 1, step = 0.05), 
                          'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1, step = 0.05),
                          'seed': trial.suggest_int('seed', 1, 1000)
                         }
    
    ## Building the model
    xgb_md = XGBClassifier(**xgboost_param_grid, n_jobs = -1, booster = 'gbtree', tree_method = 'hist').fit(X_train, Y_train)
    
    ## Predicting on the test data-frame
    xgb_md_preds = xgb_md.predict_proba(X_validation)
    
    ## Evaluating model performance on the test set
    xgb_md_error = log_loss(Y_validation, xgb_md_preds)
    
    return xgb_md_error

def lgbm_reg_objective(trial):
    
    ## Defining the LGB hyper-parameter grid
    LGB_param_grid = {'boosting_type': 'dart',
                      'n_estimators': trial.suggest_int('n_estimators', 100, 1500, 100),
                      'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, step = 0.01),
                      'num_leaves': trial.suggest_int('num_leaves', 5, 40, step = 1),
                      'max_depth': trial.suggest_int('max_depth', 3, 12),
                      'subsample': trial.suggest_float('subsample', 0.7, 1, step = 0.01), 
                      'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1, step = 0.01),
                      'random_state': trial.suggest_int('random_state', 1, 1000),
                      'reg_alpha': trial.suggest_float('reg_alpha', 0.001, 0.1, step = 0.001),
                      'reg_lambda': trial.suggest_float('reg_lambda', 0.001, 0.1, step = 0.001), 
                      'objective': 'rmse', 
                      'verbosity': -1
                     }
                     
    ## Building the LightGBM model
    model = LGBMClassifier(**LGB_param_grid, n_jobs = -1).fit(X_train, Y_train)
        
    ## Predicting on the test data-frame
    lgbm_md_preds = model.predict_proba(X_validation)
    
    ## Evaluating model performance on the test set
    lgbm_md_error = log_loss(Y_validation, lgbm_md_preds)
    
    return lgbm_md_error

def hist_objective(trial):

    ## Defining the XGBoost hyper-parameter grid
    hist_param_grid = {'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, step = 0.01),
                       'max_iter': trial.suggest_int('n_estimators', 100, 1000, 50),
                       'max_depth': trial.suggest_int('max_depth', 3, 15), 
                       'l2_regularization': trial.suggest_float('l2_regularization', 0, 0.1, step = 0.002),
                       'random_state': trial.suggest_int('random_state', 1, 500),
                      }
    
    ## Building the model
    hist_md = HistGradientBoostingClassifier(**hist_param_grid, loss = 'log_loss', early_stopping = True).fit(X_train, Y_train)
    
    ## Predicting on the test data-frame
    hist_md_preds = hist_md.predict_proba(X_validation)
    
    ## Evaluating model performance on the test set
    hist_md_error = log_loss(Y_validation, hist_md_preds)
    
    return hist_md_error

In [159]:
## Starting RandomForest
## ----
## Creating a study object and to optimize the home objective function
study_rf = optuna.create_study(direction = 'minimize')
study_rf.optimize(rf_objective, n_trials = 5)

## Starting XGBoost
## ----
## Creating a study object and to optimize the home objective function
study_xgb = optuna.create_study(direction = 'minimize')
study_xgb.optimize(xgb_objective, n_trials = 5)

## Starting LightGBM
## ----
## Creating a study object and to optimize the home objective function
study_lgbm = optuna.create_study(direction = 'minimize')
study_lgbm.optimize(lgbm_objective, n_trials = 5)

## Starting HistGradientBoosting
## ----
## Creating a study object and to optimize the home objective function
study_hist = optuna.create_study(direction = 'minimize')
study_hist.optimize(hist_objective, n_trials = 5)

## Printing best hyper-parameter set
print('Random Forest: \n', study_rf.best_trial.params)
print(study_rf.best_trial.value)

## Printing best hyper-parameter set
print('HistGB: \n', study_hist.best_trial.params)
print(study_hist.best_trial.value)

## Printing best hyper-parameter set
print('\nXGBoost: \n', study_xgb.best_trial.params)
print(study_xgb.best_trial.value)

## Printing best hyper-parameter set
print('\nLightGBM: \n', study_lgbm.best_trial.params)
print(study_lgbm.best_trial.value)

[I 2023-04-12 18:39:44,393] A new study created in memory with name: no-name-0dc032ee-6c4a-4f53-b204-18cdbffa18cf
[I 2023-04-12 18:39:47,464] Trial 0 finished with value: 0.8991340780390265 and parameters: {'n_estimators': 550, 'max_depth': 11, 'min_samples_split': 11, 'min_samples_leaf': 8, 'random_state': 22, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.8991340780390265.
[I 2023-04-12 18:39:49,020] Trial 1 finished with value: 0.9663660723318408 and parameters: {'n_estimators': 450, 'max_depth': 5, 'min_samples_split': 4, 'min_samples_leaf': 5, 'random_state': 182, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.8991340780390265.
[I 2023-04-12 18:39:59,305] Trial 2 finished with value: 0.8662066567741987 and parameters: {'n_estimators': 650, 'max_depth': 8, 'min_samples_split': 13, 'min_samples_leaf': 5, 'random_state': 327, 'max_features': None}. Best is trial 2 with value: 0.8662066567741987.
[I 2023-04-12 18:40:14,693] Trial 3 finished with value: 0.85655279049196

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2], got ['high' 'low' 'medium']

#### Modelling

In [21]:
## Reading the data
train = pd.read_csv('Data/Training.csv')
val = pd.read_csv('Data/Validation.csv')
test = pd.read_csv('Data/Testing.csv')

## Defining the input and target variables
X_train = train.drop(columns = ['interest_level']); Y_train = train['interest_level']
X_validation = val.drop(columns = ['interest_level']); Y_validation = val['interest_level']
X_test = test.drop(columns = ['ID'])

#### Modelling: One vs. Rest Classifier

Three-Layer Stacking Approach: 
- Building a series of models
- Building a few meta learners
- Averaging the meta learner predictions

In [22]:
## Building the multi-classifier (using LGBM)
one_vs_all_lgbm = OneVsRestClassifier(estimator = LGBMClassifier()).fit(X_train, Y_train)

## Predicting on the test
train_preds1 = one_vs_all_lgbm.predict_proba(X_train)
val_preds1 = one_vs_all_lgbm.predict_proba(X_validation)
test_preds1 = one_vs_all_lgbm.predict_proba(X_test)

## Predictions to data-frames
train_preds1 = pd.DataFrame(train_preds1, columns = ['high', 'low', 'medium'])
val_preds1 = pd.DataFrame(val_preds1, columns = ['high', 'low', 'medium'])
test_preds1 = pd.DataFrame(test_preds1, columns = ['high', 'low', 'medium'])

## Printing performance on the training set
train_error = log_loss(Y_train, train_preds1)
val_error = log_loss(Y_validation, val_preds1)
print(f'Training Error: {train_error}')
print(f'Validation Error: {val_error}')

Training Error: 0.5930942457035152
Validation Error: 0.830385528347405


In [23]:
## Building the multi-classifier (using XGB)
one_vs_all_xgb = OneVsRestClassifier(estimator = XGBClassifier()).fit(X_train, Y_train)

## Predicting on the test
train_preds2 = one_vs_all_xgb.predict_proba(X_train)
val_preds2 = one_vs_all_xgb.predict_proba(X_validation)
test_preds2 = one_vs_all_xgb.predict_proba(X_test)

## Predictions to data-frames
train_preds2 = pd.DataFrame(train_preds2, columns = ['high', 'low', 'medium'])
val_preds2 = pd.DataFrame(val_preds2, columns = ['high', 'low', 'medium'])
test_preds2 = pd.DataFrame(test_preds2, columns = ['high', 'low', 'medium'])

## Printing performance on the training set
train_error = log_loss(Y_train, train_preds2)
val_error = log_loss(Y_validation, val_preds2)
print(f'Training Error: {train_error}')
print(f'Validation Error: {val_error}')

Training Error: 0.4473390092025334
Validation Error: 0.8791445647990538


In [36]:
## Building the multi-classifier (using RF)
one_vs_all_rf = OneVsRestClassifier(estimator = RandomForestClassifier(max_depth = 6, n_estimators = 500)).fit(X_train, Y_train)

## Predicting on the test
train_preds3 = one_vs_all_rf.predict_proba(X_train)
val_preds3 = one_vs_all_rf.predict_proba(X_validation)
test_preds3 = one_vs_all_rf.predict_proba(X_test)

## Predictions to data-frames
train_preds3 = pd.DataFrame(train_preds3, columns = ['high', 'low', 'medium'])
val_preds3 = pd.DataFrame(val_preds3, columns = ['high', 'low', 'medium'])
test_preds3 = pd.DataFrame(test_preds3, columns = ['high', 'low', 'medium'])

## Printing performance on the training set
train_error = log_loss(Y_train, train_preds3)
val_error = log_loss(Y_validation, val_preds3)
print(f'Training Error: {train_error}')
print(f'Validation Error: {val_error}')

Training Error: 0.8731285428833246
Validation Error: 0.912021513788403


In [39]:
## Building the multi-classifier (using Hist)
one_vs_all_hist = OneVsRestClassifier(estimator = HistGradientBoostingClassifier()).fit(X_train, Y_train)

## Predicting on the test
train_preds4 = one_vs_all_hist.predict_proba(X_train)
val_preds4 = one_vs_all_hist.predict_proba(X_validation)
test_preds4 = one_vs_all_hist.predict_proba(X_test)

## Predictions to data-frames
train_preds4 = pd.DataFrame(train_preds4, columns = ['high', 'low', 'medium'])
val_preds4 = pd.DataFrame(val_preds4, columns = ['high', 'low', 'medium'])
test_preds4 = pd.DataFrame(test_preds4, columns = ['high', 'low', 'medium'])

## Printing performance on the training set
train_error = log_loss(Y_train, train_preds4)
val_error = log_loss(Y_validation, val_preds4)
print(f'Training Error: {train_error}')
print(f'Validation Error: {val_error}')

Training Error: 0.5975117780536715
Validation Error: 0.8330504736468287


In [41]:
## Building the multi-classifier (using Ada)
one_vs_all_ada = OneVsRestClassifier(estimator = AdaBoostClassifier()).fit(X_train, Y_train)

## Predicting on the test
train_preds5 = one_vs_all_ada.predict_proba(X_train)
val_preds5 = one_vs_all_ada.predict_proba(X_validation)
test_preds5 = one_vs_all_ada.predict_proba(X_test)

## Predictions to data-frames
train_preds5 = pd.DataFrame(train_preds5, columns = ['high', 'low', 'medium'])
val_preds5 = pd.DataFrame(val_preds5, columns = ['high', 'low', 'medium'])
test_preds5 = pd.DataFrame(test_preds5, columns = ['high', 'low', 'medium'])

## Printing performance on the training set
train_error = log_loss(Y_train, train_preds5)
val_error = log_loss(Y_validation, val_preds5)
print(f'Training Error: {train_error}')
print(f'Validation Error: {val_error}')

Training Error: 1.087658151805645
Validation Error: 1.0882110031313714


In [ ]:
train_preds1

In [56]:
estimators = [('rf', RandomForestClassifier(max_depth = 6, n_estimators = 500)), ('lgbm', LGBMClassifier()), 
              ('xgb', XGBClassifier()), ('hist', HistGradientBoostingClassifier()), ('ada', AdaBoostClassifier())]

stacker1 = StackingClassifier(estimators = estimators, final_estimator = RandomForestClassifier()).fit(X_train, Y_train)
stacker2 = StackingClassifier(estimators = estimators, final_estimator = LGBMClassifier()).fit(X_train, Y_train)
stacker3 = StackingClassifier(estimators = estimators, final_estimator = HistGradientBoostingClassifier()).fit(X_train, Y_train)

print(stacker1.score(X_train, Y_train))
print(stacker1.score(X_validation, Y_validation))

print(stacker2.score(X_train, Y_train))
print(stacker2.score(X_validation, Y_validation))

print(stacker3.score(X_train, Y_train))
print(stacker3.score(X_validation, Y_validation))

0.7128888888888889
0.5997968511934992
0.7146666666666667
0.5921787709497207
0.7055238095238096
0.5840528186896902


In [57]:
preds1 = pd.DataFrame(stacker1.predict_proba(X_test), columns = ['high', 'low', 'medium'])
preds2 = pd.DataFrame(stacker2.predict_proba(X_test), columns = ['high', 'low', 'medium'])
preds3 = pd.DataFrame(stacker3.predict_proba(X_test), columns = ['high', 'low', 'medium'])

In [59]:
## Configuring submission
sub[['high', 'medium', 'low']] = preds1[['high', 'medium', 'low']]
sub.to_csv('Submissions/stacker1.csv', index = False)

sub[['high', 'medium', 'low']] = preds2[['high', 'medium', 'low']]
sub.to_csv('Submissions/stacker2.csv', index = False)

sub[['high', 'medium', 'low']] = preds3[['high', 'medium', 'low']]
sub.to_csv('Submissions/stacker3.csv', index = False)

In [62]:
avg_preds = (preds2 + preds3) / 2
sub[['high', 'medium', 'low']] = avg_preds[['high', 'medium', 'low']]
sub.to_csv('Submissions/avg_stacker.csv', index = False)

#### Modelling: One vs. One Classifier

In [ ]:
## Building the multi-classifier (using RF)
one_vs_one_RF = OneVsOneClassifier(estimator = RandomForestClassifier(n_estimators = 500, max_depth = 3)).fit(X_train, Y_train)

## Predicting on the test
one_vs_one_RF_pred = one_vs_one_RF.predict(X_test)



## Configuring submission
sub[['high', 'medium', 'low']] = test_preds[['high', 'medium', 'low']]
sub.to_csv('Submissions/LGBM_2.csv', index = False)

#### Principal Component Analysis Work

#### Constructing Neural Networks